In [1]:
# 4/4/2024

# Q1 i) Generate Word Context matrix for a given Corpus on a specifed window Size
#   ii) Find Similarity b/w two words using Word Content matrix

# Q2 i) Generate word vectors using word to Vec library on IMDB data set.
#   ii) Implement text classification Using Word2Vec and LSTM.

# Q3 i) Generate word embeddings using BERT library on IMDB dataset.
#   ii) Implement text classification Using BERT embeddings and BERT classifier.


In [7]:
import numpy as np

In [8]:
def preprocess(corpus):
    new_corpus = []
    for i,docs in enumerate(corpus):
        corpus[i]="".join(ch for ch in docs if ch.isalnum() or ch ==' ')
        new_corpus.append([w.lower() for w in corpus[i].split()])
    return new_corpus

In [9]:
def getVocab(corpus):
    vocab = set()
    for c in corpus:
        vocab.update(c)
    v = {words:i for i,words in enumerate(vocab)}
    return v
        

In [10]:
corpus = [
    "I I like to eat apples and bananas.",
    "Apples are delicious fruits.",
    "Bananas are yellow in color.",
    "I like to code in Python."
]

In [11]:
def word_content_matrix(corpus,window_size):
    corpus = preprocess(corpus)
    vocab = getVocab(corpus)
#     print(corpus,vocab)
    
    word_list_len = len(vocab)
    
    word_context_matrix = np.zeros((word_list_len, word_list_len))
    
    for target_word,row in vocab.items():
#         print(target_word,row)
        for docs in corpus:
            for i,word1 in enumerate(docs):
                if word1 == target_word:
                    for j in range (i+1, min(i+window_size+1, len(docs))):
                        word_context_matrix[row][vocab[docs[j]]]+=1
                    for j in range (max(i-window_size,0),i):
                        word_context_matrix[row][vocab[docs[j]]]+=1
                        
    return word_context_matrix,vocab

In [12]:
word_context_matrix,vocab = word_content_matrix(corpus,2)
word_context_matrix

array([[0., 0., 2., 0., 3., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0.],
       [0., 0., 0., 0., 0., 1., 1., 0., 1., 0., 0., 0., 0., 0., 1.],
       [2., 0., 0., 0., 2., 0., 1., 0., 0., 1., 0., 1., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 1., 0.],
       [3., 0., 2., 0., 2., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 1., 1., 1., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0.],
       [0., 1., 0., 1., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 1.],
       [0., 0., 1., 0., 0., 0., 0., 1., 1., 0., 1., 0., 1., 0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1.],
       [1., 0., 1., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0.

In [13]:
vocab

{'like': 0,
 'bananas': 1,
 'to': 2,
 'delicious': 3,
 'i': 4,
 'and': 5,
 'apples': 6,
 'python': 7,
 'are': 8,
 'in': 9,
 'color': 10,
 'eat': 11,
 'code': 12,
 'fruits': 13,
 'yellow': 14}

In [14]:
import pandas as pd
temp_mat = []
    
for v,i in vocab.items():
    counts = {v2: 0 for v2 in vocab.items()}
    for v2,j  in vocab.items():
        counts[v2] = word_context_matrix[i][j]
    temp_mat.append(counts)

df = pd.DataFrame(temp_mat,columns=vocab.keys(),index=vocab.keys())
df

,like,bananas,to,delicious,i,and,apples,python,are,in,color,eat,code,fruits,yellow
like,0.0,0.0,2.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
bananas,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
to,2.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
delicious,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
i,3.0,0.0,2.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
and,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
apples,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
python,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
are,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0
in,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0


In [15]:
def word_similarity(word1, word2, vocab, word_content_matrix):
    if word1 not in vocab or word2 not in vocab:
        return "One or both words not found in the corpus."
    
    index1 = vocab[word1]
    index2 = vocab[word2]
    
    similarity = np.dot(word_content_matrix[index1], word_content_matrix[index2]) / (np.linalg.norm(word_content_matrix[index1]) * np.linalg.norm(word_content_matrix[index2]))
    
    return similarity


In [16]:
word_similarity('apples','bananas',vocab,word_context_matrix)

0.4082482904638631